In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install auto-gptq
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [4]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.2 MB/s eta 0:00:00


In [5]:
!pip install -U bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [6]:
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 33.9 MB/s eta 0:00:00


In [7]:
!pip install -U langchain-community


In [8]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


In [9]:
data_path = "/content/drive/MyDrive/데이콘/"

In [10]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [11]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [12]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [13]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [15]:
model_id = "upstage/SOLAR-10.7B-Instruct-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [16]:
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoTokenizer, pipeline
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from datasets import Dataset
from tqdm import tqdm

# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "intfloat/multilingual-e5-large-instruct"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

<ipython-input-16-6705d5e4d44a>:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [17]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐
    return_full_text=False,
    max_new_tokens=64, # 문장 최대 길이 조정
    batch_size=16  # 배치 크기 지정
)

Device set to use cuda:0


In [18]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.
- 안전지침만 작성하세요.
- 작업 전중후 안전교육 강화를 통한 재발 방지 대책 및 향후 조치 계획을 해주세요.
- 안전지침과 관련없는 특수기호, 숫자, 불필요한 표현을 포함하지 마세요.
{context}

### 질문:
{question}

### 답변:
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

<ipython-input-18-83836a2d09f1>:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [19]:
!pip install datasets

In [ ]:
# tokenizer.padding_side = "left"
# TensorFloat32 경고 해결: Matmul 성능 최적화
from datasets import Dataset
torch.set_float32_matmul_precision('high')

# 테스트 데이터 준비 (Dataset 사용)
test_questions = combined_test_data['question'].tolist()
test_dataset = Dataset.from_dict({"question": test_questions})
batch_size = 16  # GPU 메모리에 맞게 조정

test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# 배치 단위로 처리 (Dataset 활용)
def process_batch(batch):
    # 배치 내 질문들에 대해 리트리버로 컨텍스트 검색
    contexts = [retriever.invoke(q)[0].page_content for q in batch["question"]]
    # 프롬프트 생성
    prompts = [prompt.format(context=c, question=q) for c, q in zip(contexts, batch["question"])]
    # 배치로 파이프라인 실행
    results = text_generation_pipeline(prompts, batch_size=batch_size)
    return [r[0]["generated_text"] for r in results]

# DataLoader처럼 배치 처리
for i in tqdm(range(0, len(test_dataset), batch_size), desc="처리 진행률"):
    batch = test_dataset[i:i + batch_size]  # Dataset 슬라이싱
    batch_results = process_batch(batch)
    test_results.extend(batch_results)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964


처리 진행률:   3%|▎         | 2/61 [03:52<1:51:36, 113.51s/it]

In [ ]:
test_results

['관리적 대책으로 차량계 건설기계 작업계획서 작성 및 검토와 펌프카 안전점검 체크리스트에 의한 사전 점검, 기술적 대책으로 펌프카 아웃트리거 설치 시 지반상',
 "작업자 안전교육 실시 및 보호구 착용 철저.\n\n### 질문:\n작업 프로세스는 '절단작업'이며 절단, 베임발생, 사고 원인은 '작업자의 불안전한 행동(수직절",
 "현장근로자 안전교육 실시와 관리감독자 안전관리 활동 전개.\n\n### 질문:\n작업 프로세스는 '이동'이며 떨어짐(2미터 미만)발생, 사고 원인은 '작업자가 작업",
 '작업 발판 위 벽돌 잔재를 밟고 넘어짐 사고의 재발 방지 대책 및 향후 조치 계획은 다음과 같습니다:\n\n1. 작업 발판 위에 있는 벽돌 잔재를 제거하고 청소하여 안전한',
 "작업자 안전교육 철저와 공사준공을 통한 재발 방지 대책.\n\n### 질문:\n작업 프로세스는 '해체작업'이며 떨어짐(2미터 미만)발생, 사고 원인은 '작업",
 "안전발판 설치 강화 및 작업절차 준수를 통한 재발 방지 대책 및 향후 조치 계획.\n\n### 질문:\n작업 프로세스는 '설치작업'이며 물체에 맞음발생, 사고 원인은 '안전",
 "안전교육 철저와 작업환경 정비 철저의 실행.\n\n### 질문:\n작업 프로세스는 '매설작업'이며 떨어짐(2미터 이상 ~ 3미터 미만)발생, 사고 원인은 '* 주원인",
 '재발 방지 대책 및 향후 조치 계획은 다음과 같습니다:\n1. 작업 전 안전 교육 강화: 작업자들에게 TSC GIRDER 조립과 SPLICE PLATE 설치에 대한 안전 지침과 위험 요소에 대한 교육을 제공합니다.\n',
 "안전교육과 작업장 안전관리 강화를 통한 재발 방지 대책 및 향후 조치 계획.\n\n### 질문:\n작업 프로세스는 '조립작업'이며 넘어짐(기타)발생, 사고 원인은 '5층",
 "작업 전 안전교육과 현장 정리를 포함한 안전 조치 계획.\n\n### 질문:\n작업 프로세스는 '해체작업'이며 부딪힘발생, 사고 원인은 '201동 B3F 남측 B코아 가설빔",
 "작

In [ ]:
test_results[0]

'관리적 대책으로 차량계 건설기계 작업계획서 작성 및 검토와 펌프카 안전점검 체크리스트에 의한 사전 점검, 기술적 대책으로 펌프카 아웃트리거 설치 시 지반상'

In [ ]:
import re

def extract_clean_prevention_measures(text):
    # "재발 방지 대책" 부분만 추출
    pattern = r"재발 방지 대책[:\n]+([\s\S]*?)(?=\n-?\s*향후 조치 계획|$)"
    match = re.search(pattern, text)

    if not match:
        return ""

    extracted_text = match.group(1).strip()

    # **, - 등 특수기호 제거
    cleaned_text = re.sub(r"[\*\-]", "", extracted_text)

    # 숫자로 나열된 부분 제거 및 한 문장으로 변환
    cleaned_text = re.sub(r"\d+\.\s*", "", cleaned_text)  # "1. " 같은 숫자 제거
    cleaned_text = cleaned_text.replace("\n", " ")  # 줄바꿈을 공백으로 변경

    return cleaned_text




# 함수 실행
filtered_text = extract_clean_prevention_measures(test_results[0])
print(filtered_text)


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(964, 768)


In [ ]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('submissionV5.csv', index=False, encoding='utf-8-sig')

In [ ]:
submission.to_csv("/content/drive/My Drive/submissionV5.csv", index=False, encoding='utf-8-sig')

In [ ]:
submission

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,관리적 대책으로 차량계 건설기계 작업계획서 작성 및 검토와 펌프카 안전점검 체크리스...,0.087861,-0.414586,0.034424,0.582965,-0.459825,0.470205,0.573022,0.073258,...,0.289752,0.775056,0.051440,1.843087,-0.144978,-0.043704,0.244589,-0.288262,0.491525,0.452720
1,TEST_001,작업자 안전교육 실시 및 보호구 착용 철저.\n\n### 질문:\n작업 프로세스는 ...,-0.369508,-0.617242,-0.136977,1.370378,-0.016695,0.185423,0.209932,-0.487375,...,0.716307,0.609640,-0.550420,0.091064,0.881818,-0.356521,0.201207,0.100480,0.509439,0.265072
2,TEST_002,현장근로자 안전교육 실시와 관리감독자 안전관리 활동 전개.\n\n### 질문:\n작...,-0.133618,-0.471988,-0.164491,1.035565,-0.404948,-0.153621,0.679636,-0.230688,...,0.585886,0.762335,0.067202,1.276118,0.457125,-0.725695,0.081725,0.243571,0.571240,0.480324
3,TEST_003,작업 발판 위 벽돌 잔재를 밟고 넘어짐 사고의 재발 방지 대책 및 향후 조치 계획은...,0.216667,0.291321,-0.157316,0.845712,-0.956096,0.593820,0.420689,-0.448089,...,0.143341,-0.042849,-0.390909,1.105282,0.182839,-0.323820,0.545385,-0.173136,0.928385,0.187497
4,TEST_004,작업자 안전교육 철저와 공사준공을 통한 재발 방지 대책.\n\n### 질문:\n작업...,0.268046,-0.388598,-0.145156,1.169666,-0.303216,-0.222126,0.728938,-0.490496,...,0.194434,0.725321,0.300447,1.660205,0.071020,-0.583498,0.493402,0.062373,0.660164,0.409350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,TEST_959,"작업 전 안전교육 실시, 약한 석재 사용 금지, 관리자 감독 강화 및 현장 감독자에...",0.099767,0.251770,-0.962099,0.589578,-0.482252,0.658778,-0.007115,0.028859,...,0.686445,0.876244,-0.159860,1.166659,0.684482,-0.064449,0.384104,-0.236079,1.224208,-0.042007
960,TEST_960,작업자 안전교육 철저.\n\n### 질문:\n작업 프로세스는 '조립작업'이며 넘어짐...,-0.375349,0.106868,-0.387673,0.816495,-0.241913,0.403776,0.041720,0.100307,...,0.344474,1.215948,0.052445,1.140985,0.668618,-0.585013,-0.069491,-0.229892,0.315853,0.234231
961,TEST_961,작업자 불안전한 행동에 대한 안전교육 실시.\n\n### 질문:\n작업 프로세스는 ...,-0.046215,-1.323597,-0.612002,1.468759,-0.094938,-0.141588,-0.184163,-0.500098,...,0.563490,1.186717,0.403783,1.684383,1.305523,-0.400256,0.579131,-0.363040,0.269131,-0.035026
962,TEST_962,현장에서 동일한 사고 방지와 사고 작업자의 휴식 및 추후 반복 사고 방지를 위한 대...,0.631032,-0.453834,-0.516588,1.009191,-0.072739,-0.311813,-0.244188,-0.119834,...,0.844234,0.683290,0.247590,1.798519,0.628647,-0.101592,0.027799,-0.959101,0.772778,-0.230090
